In [ ]:
!pip install elasticsearch pandas numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.5/960.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded= files.upload()

In [ ]:
import pandas as pd

df=pd.read_csv('/content/water.csv')

In [ ]:
es= Elasticsearch(
    "https://551bc42f57554bbf8403d2eb.europe-west9.gcp.elastic-cloud.com:443",  #cluster URL
    basic_auth=("elastic", "7vUx1Dv7ym2IYNAvLYTprY8j") # elastic cloud username and password
)
# test the connetion
print(es.info)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from elasticsearch import Elasticsearch, helpers

#load the dataset
df= pd.read_csv('/content/water.csv')

#keep only necessary columns
df=df[['SITE_NAME','DATA_URL']]
df.dropna(inplace=True) #remove null values


In [ ]:
INDEX_NAME="water_sites"

#define index structure
mapping= {
    "mappings": {
        "properties" :{
            "SITE_NAME":{"type":"text"},
            "DATA_URL":{"type":"text"}
        }
    }
}

#delete the index if it exists
if es.indices.exists(index=INDEX_NAME):
  es.indices.delete(index=INDEX_NAME)

#create new index
es.indices.create(index=INDEX_NAME, body=mapping)

In [ ]:
def generate_data(df):
  for i, row in df.iterrows():
    yield{
        "_index": INDEX_NAME,
        "_id": i,
        "_source":{
            "SITE_NAME": row["SITE_NAME"],
            "DATA_URL": row["DATA_URL"]
        }
    }

    #bulk index documents
    helpers.bulk(es, generate_data(df))
    print("data uploaded successfully")

In [ ]:
vectorizer= TfidfVectorizer()
x=vectorizer.fit_transform(df['DATA_URL']) #vectorize URLs

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def search_similar_sites(query, top_n=10):
  #convert query to vector
  query_vector=vectorizer.fit_transform([query])
  #compte cosine similarity
  similarities= cosine_similarity(query_vector,x).flatten()
  #get top n results
  top_indices=similarities.argsort()[-top_n][::-1]
  results=df.iloc[top_indices][['SITE_NAME','DATA_URL']]
  return results

In [ ]:
def test_search():
  while True:
    query=input("\nEnter your search query (or type 'exit' to quit): ")
    if query.lower()=='exit':
      break
    #get the most similar sites
    results= search_similar_sites(query)
    #display result
    if results.empty:
      print("No matching results found.")
    else:
      print("\nTop results:")
      for idx, row in results.iterrows():
        print(f"-{row['SITE_NAME']}: {row['DATA_URL']}")
#run the test
test_search()

In [ ]:
!pip install gradio
import gradio as gr

def search_interface(query):
  results=search_similar_sites(query)
  if results.empty:
    return "❌ No results found."
    #format results as a markdown table
    output="| 🔹 Site Name | 🔗 Data URL |\n|---|---|\n"
    for _, row in results.iterrows():
      output += f"| **{row['SITE_NAME']}** | [Visit Site]({row['DATA_URL']}) |\n"
    return output

  #launch Gradio UI
  gr.Interface(
      fn=search_interface,
      inputs=gr.Textbox(placeholder="Enter your search query..."),
      outputs=gr.Markdown(),
      title="🌊 Water Search Engine",
      description="🔍 Enter a query to find the most relevant websites about water resources.",
      theme="compact"
  ).launch()